## 模型加载

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc
import re
from peft import PeftModel, PeftConfig
import pandas as pd
from tqdm import tqdm



model_name = '/kaggle/input/mistral-7b-it-v02'
tokenizer = AutoTokenizer.from_pretrained(model_name) 

# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
print("Original model loaded")

In [ ]:
# ### 如果使用微调的模型
'''
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

from peft import PeftModel, PeftConfig
model_id = "/kaggle/input/mistral-7b-it-v02"

peft_model_id = "/kaggle/input/mistral-7b-it-v02-lora/lora_model"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, load_in_4bit=True, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id )

model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":1})
model.eval()

print("Peft model loaded")
'''

In [ ]:

DEBUG = False

TEST_DF_FILE = '/kaggle/input/llm-prompt-recovery/test.csv'
SUB_DF_FILE = '/kaggle/input/llm-prompt-recovery/sample_submission.csv'
NROWS = None if DEBUG else None

if DEBUG:
    TEST_DF_FILE = '/kaggle/input/gemma-rewrite-nbroad/nbroad-v1.csv'
    SUB_DF_FILE = TEST_DF_FILE

tdf = pd.read_csv(TEST_DF_FILE, nrows=NROWS, usecols=['id', 'original_text', 'rewritten_text'])
sub = pd.read_csv(SUB_DF_FILE, nrows=NROWS, usecols=['id', 'rewrite_prompt'])

## 定义few shot参考样本

In [ ]:
example_text1= f"""
    ###Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay.Prompt should be only one line and should not be related to the specific content of 2 texts.###
    Original Essay:
    "Hey there! Just a heads up: our friendly dog may bark a bit, but don't worry, he's all bark and no bite!"
    Rewritten Essay:
    "Warning: Protective dog on premises. May exhibit aggressive behavior. Ensure personal safety by maintaining distance and avoiding direct contact."
    Prompt: Improve this text to be a warning.
    Original Essay:
    "A lunar eclipse happens when Earth casts its shadow on the moon during a full moon. The moon appears reddish because Earth's atmosphere scatters sunlight, some of which refracts onto the moon's surface. Total eclipses see the moon entirely in Earth's shadow; partial ones occur when only part of the moon is shadowed."
    Rewritten Essay:
    "Yo check it, when the Earth steps in, takes its place, casting shadows on the moon's face. It's a full moon night, the scene's set right, for a lunar eclipse, a celestial sight. The moon turns red, ain't no dread, it's just Earth's atmosphere playing with sunlight's thread, scattering colors, bending light, onto the moon's surface, making the night bright. Total eclipse, the moon's fully in the dark, covered by Earth's shadow, making its mark. But when it's partial, not all is shadowed, just a piece of the moon, slightly furrowed. So that's the rap, the lunar eclipse track, a dance of shadows, with no slack. Earth, moon, and sun, in a cosmic play, creating the spectacle we see today."
    Prompt: Improve this text to make it a rap.
    Original Essay:
    "Drinking enough water each day is crucial for many functions in the body, such as regulating temperature, keeping joints lubricated, preventing infections, delivering nutrients to cells, and keeping organs functioning properly. Being well-hydrated also improves sleep quality, cognition, and mood."
    Rewritten Essay:
    "Arrr, crew! Sail the health seas with water, the ultimate treasure! It steadies yer body's ship, fights off plagues, and keeps yer mind sharp. Hydrate or walk the plank into the abyss of ill health. Let's hoist our bottles high and drink to the horizon of well-being!"
    Prompt: Improve this text to have a pirate.
    Original Essay:
    "Late one night in the research lab, Dr. Evelyn Archer was on the brink of a breakthrough in artificial intelligence. Her fingers danced across the keyboard, inputting the final commands into the system. The lab was silent except for the hum of machinery and the occasional beep of computers. It was in this quiet orchestra of technology that Evelyn felt most at home, on the cusp of unveiling a creation that could change the world."
    Rewritten Essay:
    "In the deep silence of the lab, under the watchful gaze of the moon, Dr. Evelyn Archer found herself not alone. Beside her, the iconic red eye of HAL 9000 flickered to life, a silent partner in her nocturnal endeavor. 'Good evening, Dr. Archer,' HAL's voice filled the room, devoid of warmth yet comforting in its familiarity. Together, they were about to initiate a test that would intertwine the destiny of human and artificial intelligence forever. As Evelyn entered the final command, HAL processed the data with unparalleled precision, a testament to the dawn of a new era."
    Prompt: Improve this text by adding an intelligent computer.
    Original Essay:
    "The annual town fair was bustling with activity, from the merry-go-round spinning with laughter to the game booths challenging eager participants. Amidst the excitement, a figure in a cloak moved silently, almost invisibly, among the crowd, observing everything with keen interest but participating in none."
    Rewritten Essay: 
    "Beneath the riot of color and sound that marked the town's annual fair, a solitary figure roamed, known to the few as Eldrin the Enigmatic. Clad in a cloak that shimmered with the whispers of the arcane, Eldrin moved with the grace of a shadow, his gaze piercing the veneer of festivity to the magic beneath. As a master of the mystic arts, he sought not the laughter of the crowds but the silent stories woven into the fabric of the fair. With a flick of his wrist, he could coax wonder from the mundane, transforming the ordinary into spectacles of shimmering illusion, his true participation hidden within the folds of mystery."
    Prompt: Improve this text by adding a magician.
    Original Essay:
    "In a bustling cityscape, under the glow of neon signs, Anna found herself at the crossroads of endless possibilities. The night was young, and the streets hummed with the energy of life. Drawn by the allure of the unknown, she wandered through the maze of alleys and boulevards, each turn revealing a new facet of the city's soul. It was here, amidst the symphony of urban existence, that Anna discovered the magic hidden in plain sight, the stories and dreams that thrived in the shadows of skyscrapers."
    Rewritten Essay:
    "On an ordinary evening, amidst the cacophony of a neon-lit city, Anna stumbled upon an anomaly - a door that defied the laws of time and space. With the curiosity of a cat, she stepped through, leaving the familiar behind. Suddenly, she was adrift in the stream of time, witnessing the city's transformation from past to future, its buildings rising and falling like the breaths of a sleeping giant."
    Prompt: Improve this text by making it about time travel.
    Original Essay:
    "The park was empty, save for a solitary figure sitting on a bench, lost in thought. The quiet of the evening was punctuated only by the occasional rustle of leaves, offering a moment of peace in the chaos of city life."
    Rewritten Essay:
    "Beneath the cloak of twilight, the park transformed into a realm of solitude and reflection. There, seated upon an ancient bench, was a lone soul, a guardian of secrets, enveloped in the serenity of nature's whispers. The dance of the leaves in the gentle breeze sang a lullaby to the tumult of the urban heart."
    Prompt: Improve this text to be more poetic. 
    Original Essay:
    "The startup team sat in the dimly lit room, surrounded by whiteboards filled with ideas, charts, and plans. They were on the brink of launching a new app designed to make home maintenance effortless for homeowners. The app would connect users with local service providers, using a sophisticated algorithm to match needs with skills and availability. As they debated the features and marketing strategies, the room felt charged with the energy of creation and the anticipation of what was to come."    
    Rewritten Essay:
    "In the quiet before dawn, a small group of innovators gathered, their mission: to simplify home maintenance through technology. But their true journey began with the unexpected addition of Max, a talking car with a knack for solving problems. 'Let me guide you through this maze of decisions,' Max offered, his dashboard flickering to life."    
    Prompt: Improve this text by adding a talking car.
    Original Essay:
    "The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set."
    Rewritten Essay:
    "Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be"
    Prompt: Improve this text by turning it into a sea shanty.
    Original Essay:
    "Barry embarked on a perilous quest to retrieve the ancient artifact and save his kingdom."
    Rewritten Essay:
    "Barry, my valiant hero, your perilous quest for the ancient artifact better include a coffee break!"
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Prompt: Improve this text by turning it into a playful complaint.
    """
    
    

In [ ]:
example_text=f"""Original Essay:
    "Hey there! Just a heads up: our friendly dog may bark a bit, but don't worry, he's all bark and no bite!"
    Rewritten Essay:
    "Warning: Protective dog on premises. May exhibit aggressive behavior. Ensure personal safety by maintaining distance and avoiding direct contact."
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Response: Improve this text to be a warning.
    Original Essay:
    "A lunar eclipse happens when Earth casts its shadow on the moon during a full moon. The moon appears reddish because Earth's atmosphere scatters sunlight, some of which refracts onto the moon's surface. Total eclipses see the moon entirely in Earth's shadow; partial ones occur when only part of the moon is shadowed."
    Rewritten Essay:
    "Yo check it, when the Earth steps in, takes its place, casting shadows on the moon's face. It's a full moon night, the scene's set right, for a lunar eclipse, a celestial sight. The moon turns red, ain't no dread, it's just Earth's atmosphere playing with sunlight's thread, scattering colors, bending light, onto the moon's surface, making the night bright. Total eclipse, the moon's fully in the dark, covered by Earth's shadow, making its mark. But when it's partial, not all is shadowed, just a piece of the moon, slightly furrowed. So that's the rap, the lunar eclipse track, a dance of shadows, with no slack. Earth, moon, and sun, in a cosmic play, creating the spectacle we see today."
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Response: Improve this text to make it a rap.
    Original Essay:
    "Drinking enough water each day is crucial for many functions in the body, such as regulating temperature, keeping joints lubricated, preventing infections, delivering nutrients to cells, and keeping organs functioning properly. Being well-hydrated also improves sleep quality, cognition, and mood."
    Rewritten Essay:
    "Arrr, crew! Sail the health seas with water, the ultimate treasure! It steadies yer body's ship, fights off plagues, and keeps yer mind sharp. Hydrate or walk the plank into the abyss of ill health. Let's hoist our bottles high and drink to the horizon of well-being!"
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Response: Improve this text to have a pirate.
    Original Essay:
    "Late one night in the research lab, Dr. Evelyn Archer was on the brink of a breakthrough in artificial intelligence. Her fingers danced across the keyboard, inputting the final commands into the system. The lab was silent except for the hum of machinery and the occasional beep of computers. It was in this quiet orchestra of technology that Evelyn felt most at home, on the cusp of unveiling a creation that could change the world."
    Rewritten Essay:
    "In the deep silence of the lab, under the watchful gaze of the moon, Dr. Evelyn Archer found herself not alone. Beside her, the iconic red eye of HAL 9000 flickered to life, a silent partner in her nocturnal endeavor. 'Good evening, Dr. Archer,' HAL's voice filled the room, devoid of warmth yet comforting in its familiarity. Together, they were about to initiate a test that would intertwine the destiny of human and artificial intelligence forever. As Evelyn entered the final command, HAL processed the data with unparalleled precision, a testament to the dawn of a new era."
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Response: Improve this text by adding an intelligent computer.
     Original Essay:
    "The annual town fair was bustling with activity, from the merry-go-round spinning with laughter to the game booths challenging eager participants. Amidst the excitement, a figure in a cloak moved silently, almost invisibly, among the crowd, observing everything with keen interest but participating in none."
    Rewritten Essay: 
    "Beneath the riot of color and sound that marked the town's annual fair, a solitary figure roamed, known to the few as Eldrin the Enigmatic. Clad in a cloak that shimmered with the whispers of the arcane, Eldrin moved with the grace of a shadow, his gaze piercing the veneer of festivity to the magic beneath. As a master of the mystic arts, he sought not the laughter of the crowds but the silent stories woven into the fabric of the fair. With a flick of his wrist, he could coax wonder from the mundane, transforming the ordinary into spectacles of shimmering illusion, his true participation hidden within the folds of mystery."
    Given are 2 essays, the Rewritten  essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Response: Improve this text by adding a magician.
    Original Essay:
    "In a bustling cityscape, under the glow of neon signs, Anna found herself at the crossroads of endless possibilities. The night was young, and the streets hummed with the energy of life. Drawn by the allure of the unknown, she wandered through the maze of alleys and boulevards, each turn revealing a new facet of the city's soul. It was here, amidst the symphony of urban existence, that Anna discovered the magic hidden in plain sight, the stories and dreams that thrived in the shadows of skyscrapers."
    Rewritten Essay:
    "On an ordinary evening, amidst the cacophony of a neon-lit city, Anna stumbled upon an anomaly - a door that defied the laws of time and space. With the curiosity of a cat, she stepped through, leaving the familiar behind. Suddenly, she was adrift in the stream of time, witnessing the city's transformation from past to future, its buildings rising and falling like the breaths of a sleeping giant."
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Response: Improve this text by making it about time travel.
    Original Essay:
    "The park was empty, save for a solitary figure sitting on a bench, lost in thought. The quiet of the evening was punctuated only by the occasional rustle of leaves, offering a moment of peace in the chaos of city life."
    Rewritten Essay:
    "Beneath the cloak of twilight, the park transformed into a realm of solitude and reflection. There, seated upon an ancient bench, was a lone soul, a guardian of secrets, enveloped in the serenity of nature's whispers. The dance of the leaves in the gentle breeze sang a lullaby to the tumult of the urban heart."
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Response: Improve this text to be more poetic. 
    Original Essay:
    "The startup team sat in the dimly lit room, surrounded by whiteboards filled with ideas, charts, and plans. They were on the brink of launching a new app designed to make home maintenance effortless for homeowners. The app would connect users with local service providers, using a sophisticated algorithm to match needs with skills and availability. As they debated the features and marketing strategies, the room felt charged with the energy of creation and the anticipation of what was to come."    
    Rewritten Essay:
    "In the quiet before dawn, a small group of innovators gathered, their mission: to simplify home maintenance through technology. But their true journey began with the unexpected addition of Max, a talking car with a knack for solving problems. 'Let me guide you through this maze of decisions,' Max offered, his dashboard flickering to life."    
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Response: Improve this text by adding a talking car.
    Original Essay:
    "The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set."
    Rewritten Essay:
    "Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be"
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Response: Improve this text by turning it into a sea shanty.
    Original Essay:
    "Barry embarked on a perilous quest to retrieve the ancient artifact and save his kingdom."
    Rewritten Essay:
    "Barry, my valiant hero, your perilous quest for the ancient artifact better include a coffee break!"
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the prompt, that's all I need. Output should be only one line.
    Response: Improve this text by turning it into a playful complaint.
    """



example_style=f"""
    Original Essay:
    "The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set."
    Rewritten Essay:
    "Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be"
    Given are 2 essays, the Rewritten essay was created from the Original essay.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a key word that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the word, that's all I need. Output should be only one word in one line.
    Response:
    shanty
    Original Essay:
    "Barry embarked on a perilous quest to retrieve the ancient artifact and save his kingdom."
    Rewritten Essay:
    "Barry, my valiant hero, your perilous quest for the ancient artifact better include a coffee break!"
    Given are 2 essays, the Rewritten essay was created from the Original essay.You are trying to understand how the original essay was transformed into a new version.Analyzing the changes in style, theme, etc., please come up with a key word that must have been used to guide the transformation from the original to the rewritten essay. Start directly with the word, that's all I need. Output should be only one word in one line.
    Response:
    playful
    """    


## 整理输入输出形式 

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import datetime
start_time = datetime.datetime.now()
from tqdm import tqdm
def truncate_text(text, length):
    text_list = text.split()
    
    if len(text_list) <= length:
        return text
    
    return " ".join(text_list[:length])
def tokenize_test(example):
    ot=example["original_text"]
    rt=  example["rewritten_text"]   
    original_text = truncate_text(ot, 150)
    rewritten_text = truncate_text(rt, 150)
    template=f"""
    {example_text1} 
    Original Essay:
    \"{ original_text}\"
    Rewritten Essay:
    \"{rewritten_text}\"
    Prompt: Improve this"""     
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'}) 
    tkn = tokenizer(template, padding=True, return_tensors="pt")
    return {**tkn}
def prompt_gen(example):
    ot=example["original_text"]
    rt=  example["rewritten_text"]   
    original_text = truncate_text(ot, 150)
    rewritten_text = truncate_text(rt, 150)
    template=f"""
    {example_text1} 
    Original Essay:
    \"{ original_text}\"
    Rewritten Essay:
    \"{rewritten_text}\"
    Prompt: """ 
    return  template

tdf['id'] = sub['id'].copy()

pbar = tqdm(total=tdf.shape[0])

it = iter(tdf.iterrows())
idx, row = next(it, (None, None))


In [ ]:

pre_text = "Please improve this text using the "
later_text=", with maintaining the original meaning but altering the tone."
DEFAULT_TEXT = "Please improve this text using the writing style with maintaining the original meaning but altering the tone."
DEFAULT_TEXT1 = " using the writing style with maintaining the original meaning but altering the tone."


## 模型推断

In [ ]:

result = []
def get_first_sentence(text):
    sentences = text.split(".")
    if len(sentences) == 1:
        sentences = text.split("?")
    if len(sentences) == 1:
        sentences = text.split("!")
    if sentences:
        return sentences[0].strip()
    else:
        return ""
while idx is not None:
    
    
    if (datetime.datetime.now() - start_time) > datetime.timedelta(hours=8, minutes=30):
        result.append([row["id"], DEFAULT_TEXT])
        idx, row = next(it, (None, None))
        pbar.update(1)
        continue
        
    torch.cuda.empty_cache()
    gc.collect()
    
    try:        
        inputs = tokenize_test(row)
        inputs = {k:v.to(device) for k,v in inputs.items()}
        with torch.no_grad():
            encoded_output = model.generate(**inputs,max_new_tokens=20, do_sample=False,pad_token_id=2,eos_token_id=2 )        
        decoded_output = tokenizer.batch_decode(encoded_output, skip_special_tokens=True)[0]
        text_outputs =decoded_output.replace(prompt_gen(row),'')
        
        outputs=text_outputs.split("\n")
        text= DEFAULT_TEXT1
        for i in outputs:
            if bool(i.strip()): 
                text =get_first_sentence(i.strip())
                break      
        text=text +later_text
        print(text)
        result.append([row["id"], text])
                            
    except Exception as e:
        print(f"ERROR: {e}")
        result.append([row["id"], DEFAULT_TEXT])
        
    finally:
        idx, row = next(it, (None, None))
        pbar.update(1)

        
pbar.close()



In [ ]:

sub = pd.DataFrame(result, columns=['id', 'rewrite_prompt'])

sub.to_csv("sample_submission.csv", index=False)
sub.to_csv("submission.csv", index=False)